### Creation of a NFW extended source

Our goal now is to create files that contain the photon distribution in a patch of the sky, for an extended source that is modelled as a subhalo that follows a truncated NFW. We assume thus that:

$$ \rho(r) \propto \frac{1}{r},$$

for $r< R_t$ and $\rho(r) = 0$ for $r> R_t$,

where $r$ the distance from the center of the (sub) halo and $R_t$ the truncation radius:

\begin{equation}
r(\theta, d, l) = \sqrt{d^2 + l^2 - 2dlcos\theta} \qquad (1)
\end{equation}

where $l$ is distance along the line of sight, $d$ the distance to the center of the subhalo and $\theta$ is the angle to the center of the subhalo. 

The total light at an angle $\theta$ from the center of subhalo is proportional to the $J$ factor, we define as:



$$J(\theta) \propto \int_{\mbox{l.o.s}} \rho^2[r(d,l,\theta)]\,\, dl \propto \int_{\mbox{l.o.s}} \frac{1}{r^2(d,l,\theta)} dl$$

We want to find the limits of integration along the line of sight at a given angle $\theta$. These corespond to values of $l$ for which we have that $r = R_t$. Thus, we solve:

\begin{equation}
R_t^2 = d^2 + l^2 - 2dl\cos\theta
\end{equation}

The two independent solutions are (for small values of $\theta$):

\begin{equation}
l_{\pm} = d \pm \sqrt{R_t^2 -(d \theta)^2} \qquad (2)
\end{equation}

For:
\begin{equation}
\theta < \frac{R_t}{d} \qquad (3)
\end{equation}

Actually we don't want to work with the parameters $R_t$ and $d$; we want to have - if possible - only angular parameters. So, we introduce the $\mathbf{maximum}$ angle $\theta_{max}$, defined as:

\begin{equation}
\theta_{max} \equiv \frac{R_t}{d} \qquad (4)
\end{equation}

Using this, we can write the limits of integration (2) as:

\begin{equation}
l_\pm = d \alpha_\pm, \qquad (5)
\end{equation}

where we defined:

\begin{equation}
\alpha_\pm = \left( 1 \pm \theta_{max}\sqrt{1 - \left( \frac{\theta}{\theta_{max}}\right)^2} \right) \qquad (6)
\end{equation}

Let's write again the $J$ factor (introduce a normalization constant $A$ in front).

\begin{equation}
J(\theta) = A \int_{d\alpha_-}^{d\alpha_+} \, \frac{1}{d^2+l^2-2dl\cos\theta}dl
\end{equation} 


Introduce now the parameter $u$, such as $l = u \cdot d $. Since $d$ is constant $dl = du \cdot d$ (here $du$ differential). The limits of the integral become $a_{\pm}$. so, performing this change of variables:

\begin{equation}
J(\theta) = A \int_{\alpha_-}^{\alpha_+} \frac{d}{d^2 + d^2u^2 - 2d^2u\cos\theta}du \Rightarrow \boxed{J(\theta) = \frac{A}{d} \int_{\alpha_-}^{\alpha_+} \, \frac{1}{1+u^2 - 2u \cos\theta} du \qquad (7)}
\end{equation}

Actually $A/d$ is  just an overall normalization constant. 
We have assumed that in a region of angular radius $\theta_{\max}$ is enclosed the total light of the subhalo. To get the total light we have thus to integrate (8) from $\theta = 0$ to $\theta = 5\sigma$ and then multiply by $2\pi$ since we have to sweep a full circle to cover the whole area. Thus, the normalised angular distribution of photons is given by:

\begin{equation}
\displaystyle
p(\theta) = \frac{J(\theta)}{2\pi \int_0^{\theta_{max}} \theta \, J(\theta)\, d\theta}  = \frac{J(\theta)}{2\pi \int_0^{\theta_{max}} \, d\theta \, \theta \, \frac{A}{d}\int_{\alpha_-}^{\alpha_+} \frac{1}{1+u^2-2u\cos\theta}du}  
= \frac{\int_{\alpha_-}^{\alpha_+} \frac{1}{1+u^2-2u\cos\theta}du}{2\pi \int_0^{\theta_{max}} \, d\theta \, \theta \, \int_{\alpha_-}^{\alpha_+} \frac{1}{1+u^2-2u\cos\theta}du} \qquad (8)
\end{equation}

In practice we will never use this. We will just use the integral in equation (7) to calculate the (unnormalised) value of $J(\theta)$ at each pixel. Then we will normalise by summing the values at every pixel and dividing the value at each pixel by that sum.

##### Defining $\sigma$ 

Before closing this section, let's define the parameter $\sigma$, which is the angle inside which 68$\%$ of the total light is included. Using (7) and the normalization of (8), we can define/find $\sigma$ through:

\begin{equation}
\frac{\int_{0}^{\sigma} \, d\theta \, \theta \,\int_{\alpha_-}^{\alpha_+} \frac{1}{1+u^2-2u\cos\theta}du}{ \int_{0}^{\theta_{max}} \, d\theta \, \theta \, \int_{\alpha_-}^{\alpha_+} \frac{1}{1+u^2-2u\cos\theta}du} = 0.68 \qquad (9)
\end{equation}

In our case, in order to be consistent whith what we have done so far, we will require $\sigma = 0.25^o$. 

In what follows, I will try to numerically solve eq. (10) in order to determine $\theta_{max}$ for the given $\sigma$. However, since for small theta the first integral diverges I'm not sure if this is going to work. Let's try and see if we can get a stable solution. If not we will have to determine everything numerically, by "populating" tiny pixels and taking averages.

I'll describe the process later

### Part 1: Attempt to determine $\theta_{max}$ solving numerically eq. (10)

In this part we will try to solve numerically eq. (10) and see if we can get a  $\mathit{numerically \,\,stable}$ solution for $\theta_{\max}$, using the bisection method.

In [1]:
import numpy as np
import scipy.integrate as integrate
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
# Degree to rad and rad to degree conversions

degrad = 0.0174533 #degrees to rads
raddeg = 57.29575 #rads to degrees


#-------------------------------------------------

# Now define a sigma, the sigma you want in DEGREES

sigma = 0.25

# Contvert this to rad

sigma = sigma*degrad

#-------------------------------------------------

# In this section define the bisection function

#def samesign(a,b):
#    return a*b > 0


#def bisection(func, low, high):
#    'Find root of continuous function where f(low) and f(high) have opposite signs'
    
#    assert not samesign(func(low), func(high))
    
#    for i in range(40):
#        midpoint = (low + high) / 2.0
#        if samesign(func(low), func(midpoint)):
#            low = midpoint
#        else:
#            high = midpoint
#    return midpoint
#--------------------------------------------------

# In this section define funtion for integration and limits

#Note that there are two double integrals. The limits of both integrals in u are the same, but the limits in theta are not

#def integrand(u, thet):
#    integr = 1.0/(1.0 + u**2.0 -2.0*u*np.cos(thet))
#    return integr

#bounds in u

#def alpha_up(thet,x):
#    aplus = 1 + x*np.sqrt(1.0 - (thet/x)**2.0)
#    return aplus
    

#def alpha_down(thet,x):
#    aminus = 1 - x*np.sqrt(1.0 - (thet/x)**2.0)
#    return aminus
    
#-------------------------------------------

# In this section we define the function whose root we want to find


#def func_root(x):
#    Integ1 = integrate.dblquad(lambda u, thet: integrand(u,thet), 0.0, sigma, lambda thet: alpha_down(thet,x), lambda thet: alpha_up(thet,x))
#    Integ1 = Integ1[0]
#    Integ2 = integrate.dblquad(lambda u, thet: integrand(u,thet), 0.0, x, lambda thet: alpha_down(thet,x), lambda thet: alpha_up(thet,x))
#    Integ2 = Integ2[0]
#    return Integ1/Integ2 - 0.68
#-----------------------------------------


# Now we are almost ready to perform the bisection and find the root - the angle theta_max 

# We have to define limits

#lower limit set equal to sigma. upper limit equal to 1e4 sigma

#lowlimit = sigma
#uplimit = (1e5)*sigma

#theta_max = bisection(func_root, lowlimit, uplimit)

#functval = func_root(theta_max)

# convert now theta critical to degrees

#theta_maxdeg = theta_max*raddeg

#print('Critical angle in degrees is:')
#print(theta_maxdeg)

#print('Value of the function is:')
#print(functval)


As we can see, the above procedure is not efficient; the integrals seem to diverge. We will follow a different strategy.  

We'll try to solve the whole problem totally numerically. We will take again a grid of pixels as before; but now we will further split each pixel in many smaller pixels and we will calculate the value of $J(\theta)$ at each of these smaller pixels. Then we will assign the mean value of these $J(\theta)$'s as the $J(\theta)$ of the pixel.

We have to do this for every pixel of a grid, and try different values of $\theta_{max}$ in order to find that value of $\theta_{max}$ for which the 68$\%$ of the light is inside an angular radius of $\sigma = 0.25^o$. Before solving the full problem we will start with a

### Warm up: Calculating the value of $J(\theta)$ inside a pixel at $\theta$ close to 0.

As a first exercise, we will pick a pixel of size 0.05$^o$ $\times$ 0.05$^o$. We will define $\theta = 0$ at one of its edges (similarly to what we'll have in the bigger grid) and picking a value for $\theta_{max}$. 

We will take $theta=0$ at the lower left corner of the pixel (grid) and $\theta_{max} = 10\sigma = 10 \cdot 0.25 = 2.5^o$. 

In [3]:
# first define a theta max in degrees and then convert it to radians

thet_max = 2.5
thet_max = thet_max*degrad

# define the size of the bin

binlen = 0.05
binlen = binlen*degrad 

#--------------------------------------------------------------------------
# Here I define function that takes as input the position of the origin a,b, the position of the subin and the subbin
# length - since it changes - and returns the value of j in the subbin

def integrand(x, thet):
    integr = 1.0/(1.0 + x**2.0 -2.0*x*np.cos(thet))
    return integr


def subbin_J(a,b,i,j,sublen):
    'a,b are the coordinates of the origin i,j those of the bin and sublen the size of each subbin'
    
    distx = (abs(j - b) + 0.5)*sublen
    disty = (abs(i - a) + 0.5)*sublen
    #calculate distance in theta
    
    thet = np.sqrt(distx**2.0 + disty**2.0)
    #limits of integration
    alow = 1.0 - thet_max*np.sqrt(1.0 - (thet/thet_max)**2.0)
    aup = 1.0 + thet_max*np.sqrt(1.0 - (thet/thet_max)**2.0)
    integroulino = integrate.quad(integrand, alow, aup, args=(thet))[0]
    return integroulino
    
#-----------------------------------------------------------------------------------------------
# Here I  define an 'old' starting value for J at this bin, just to compare 
oldval = 1.0
#-----------------------------------------------------------------------------------------------
# First calculation of the value of J in that bin

# Now, define a starting number of subbins, let it be N = 100
N = 50
#length -in rads- of each subbin
sublen = binlen/N

#define the matrix which contain the grid

J_matrix = np.zeros((N,N))
#coordinates of the "origin"
a = N-1
b = 0

#let's populate the matrix now 

for i in range (0,N):
    for j in range (0,N):
        J_matrix[i,j] = subbin_J(a,b,i,j,sublen)
        
newval = np.average(J_matrix)



print(newval)


6271.0539114


//anaconda/lib/python3.5/site-packages/scipy/integrate/quadpack.py:356: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)


We can see that see that a $50 \times 50$ splitting of the the pixels closest to the origin, gives an accuracy better than $1\%$ in the calculation of the mean $J(\theta)$ of that pixel.

We want to investigate if there is need to split into smaller sub-pixels the neighbouring pixels to the first ones.

In [4]:
# first define a theta max in degrees and then convert it to radians

#thet_max = 0.3
#thet_max = thet_max*degrad

# define the size of the bin

#binlen = 0.05
#binlen = binlen*degrad 

#--------------------------------------------------------------------------
# Here I define function that takes as input the position of the origin a,b, the position of the subin and the subbin
# length - since it changes - and returns the value of j in the subbin

#def integrand(x, thet):
#    integr = 1.0/(1.0 + x**2.0 -2.0*x*np.cos(thet))
#    return integr


#def subbin_J(a,b,i,j,sublen):
#    'a,b are the coordinates of the origin i,j those of the bin and sublen the size of each subbin'
    
#    distx = (abs(j - b) + 0.5)*sublen + binlen
#    disty = (abs(i - a) + 0.5)*sublen
#    #calculate distance in theta
    
#    thet = np.sqrt(distx**2.0 + disty**2.0)
#    alow = 1.0 - thet_max*np.sqrt(1.0 - (thet/thet_max)**2.0)
#    aup = 1.0 + thet_max*np.sqrt(1.0 - (thet/thet_max)**2.0)
#    integroulino = integrate.quad(integrand, alow, aup, args=(thet))[0]
#    return integroulino
    
#-----------------------------------------------------------------------------------------------
# Here I  define an 'old' starting value for J at this bin, just to compare 
#oldval = 1.0
#-----------------------------------------------------------------------------------------------
# First calculation of the value of J in that bin

# Now, define a starting number of subbins, let it be N = 100
#N = 50
#length -in rads- of each subbin
#sublen = binlen/N

#define the matrix which contain the grid

#J_matrix = np.zeros((N,N))
#coordinates of the "origin"
#a = N-1
#b = 0

#let's populate the matrix now 

#for i in range (0,N):
#    for j in range (0,N):
#        J_matrix[i,j] = subbin_J(a,b,i,j,sublen)
        
#newval = np.average(J_matrix)



#print(newval)

### Full solution of the problem - determination of $\theta_{\max}$

In [29]:
# Degree to rad and rad to degree conversions

degrad = 0.0174533 #degrees to rads
raddeg = 57.29575 #rads to degrees

#-------------------------------------------------

# Now define a sigma, the sigma you want in DEGREES

sigma = 0.25

# Contvert this to rad

sigma = sigma*degrad

#Size of the bigger grid N x N points. Define N=400
#Whenever I want to create a grid I will create a N x N grid 

N = 400

# Length of each bin of the grid in degrees = 0.05

binlen = 0.05 #in degrees
binlen = binlen*degrad

#----------------------------------------------------------------------------------------
#----------------------------------------------------------------------------------------
# In this section define the bisection function
# This function takes a function and two points - low and high - and finds the root of it

def samesign(d,e):
    return d*e > 0


def bisection(func, low, high):
    'Find root of continuous function where f(low) and f(high) have opposite signs'
    
    assert not samesign(func(low), func(high))
    
    for i in range(40):
        print("Entrance no:")
        print(i)
        midpoint = (low + high) / 2.0
        if samesign(func(low), func(midpoint)):
            low = midpoint
        else:
            high = midpoint
    return midpoint
#-----------------------------------------------------------------
#-----------------------------------------------------------------

# Here I define function that takes as input the position of the origin a,b, the position of the bin or the subbin and the subbin
# length - since it changes - and returns the value of j in the subbin. also the value of the 

def integrand(u, thet):
    integr = 1.0/(1.0 + u**2.0 - 2.0*u*np.cos(thet))
    return integr


def J_value(ori,orj,i,j, length, thet_trial):
    'ori,orj are the coordinates of the origin i,j those of the bin and len the size of each bin'
    
    distx = (abs(j - orj) + 0.5)*length
    disty = (abs(i - ori) + 0.5)*length
    #calculate distance in theta
    
    thet = np.sqrt(distx**2.0 + disty**2.0)
    if (thet < thet_trial):
        #limits of integration
        alow = 1.0 - thet_trial*np.sqrt(1.0 - (thet/thet_trial)**2.0)
        aup = 1.0 + thet_trial*np.sqrt(1.0 - (thet/thet_trial)**2.0)
        integroulino = integrate.quad(integrand, alow, aup, args=(thet))[0]
    else:
        integroulino = 0.0
    return integroulino

#-------------------------------------------------------------------
#-------------------------------------------------------------------
#The array  we create each time is a 400*400 arrray - it has no central point. Instead we will define four central points 
#define the coordinates of the four four central points. Define the points as lr = low right, ll = low left, ur = upper right
#ul = upper left. A simple diagram would be very helpful here. 
#i is row, j is column

adef = int(N/2)

lri = adef; lrj = adef
lli = adef; llj = adef - 1
uri = adef - 1; urj = adef
uli = adef -1; ulj = adef - 1

##the following function finds the quarter of the grid 
# named again ll, lr, ur, ul  


def quarter(i,j):
    if ((i >= lri) and (j >= lrj)):
        quarto = "lr"
    elif ((i >= lli) and (j <= llj)):
        quarto = "ll"
    elif ((i <= uri) and (j >= urj)):
        quarto = "ur"
    elif ((i <= uli) and (j <= ulj)):
        quarto = "ul"
    return quarto
#---------------------------------------------------------------
#---------------------------------------------------------------
# This is the main body of the program, it takes a trial value for theta max and 
# calculates the sum of values of J_theta inside sigma and the sum of all values of J_theta 
# inside the gid and returns them
# We will need also one more function in case we deal with one of the central sources


def subbin(length, thet_trial):
    'This function calculates and returns the mean value of J for the four central points. the value is the same for all'
    
    #number of subbins of teh subgrid
    numb = 50
    #length of subbins
    subleng = length/numb
    
    subgrid = np.zeros((numb,numb))
    
    #coordinates of the origin
    origi = numb - 1
    origj = 0
    
    for i in range (0,numb):
        for j in range (0,numb):
            subgrid[i][j] = J_value(origi,origj,i,j,subleng,thet_trial)
    
    sub_val = np.average(subgrid)
    return sub_val




def J_sum(thet_trial):
    'This function calculates and returns for each theta trial the value of the sum of Js and the sum inside sigma'

    grid = np.zeros((N,N))
    
    subbin_val = subbin(binlen,thet_trial)
    
    inner_J = 0.0
    for i in range (0,N):
        for j in range (0,N):
            quar = quarter(i,j)
            if (quar == "lr"):
                if ((i == lri)and(j == lrj)):
                    entry = subbin_val
                else:
                    entry = J_value(lri,lrj,i,j,binlen,thet_trial)
                    
                distx = (abs(j - lrj) + 0.5)*binlen
                disty = (abs(i - lri) + 0.5)*binlen
                #calculate distance in theta
                thet = np.sqrt(distx**2.0 + disty**2.0)
                if (thet <= sigma):
                    inner_J = inner_J + entry
                
            elif (quar == "ll"):
                if ((i == lli)and(j == llj)):
                    entry = subbin_val
                else:
                    entry = J_value(lli,llj,i,j,binlen,thet_trial)
                    
                distx = (abs(j - llj) + 0.5)*binlen
                disty = (abs(i - lli) + 0.5)*binlen
                #calculate distance in theta
                thet = np.sqrt(distx**2.0 + disty**2.0)
                if (thet <= sigma):
                    inner_J = inner_J + entry
            elif (quar == "ur" ):
                if ((i == uri)and(j == urj)):
                    entry = subbin_val
                else:
                    entry = J_value(uri,urj,i,j,binlen,thet_trial)
                    
                distx = (abs(j - urj) + 0.5)*binlen
                disty = (abs(i - uri) + 0.5)*binlen
                #calculate distance in theta
                thet = np.sqrt(distx**2.0 + disty**2.0)
                if (thet <= sigma):
                    inner_J = inner_J + entry    
                    
            elif (quar == "ul"):
                if ((i == uli)and(j == ulj)):
                    entry = subbin_val
                else:
                    entry = J_value(uli,ulj,i,j,binlen,thet_trial)
                    
                distx = (abs(j - ulj) + 0.5)*binlen
                disty = (abs(i - uli) + 0.5)*binlen
                #calculate distance in theta
                thet = np.sqrt(distx**2.0 + disty**2.0)
                if (thet <= sigma):
                    inner_J = inner_J + entry
            
            grid[i][j] = entry
    
    #sum of J_theta of the whole grid 
    tot_J = np.sum(grid)
    
    
    return inner_J, tot_J
    



#----------------------------------------------------------------
# define the function whose root we want to find

def func_root(x):
    'We will define another function which calculates the sum of values of the grid up to theta = sigma and the whole grid '
    
    upper = J_sum(x)[0]
    lower = J_sum(x)[1]
    return upper/lower - 0.68


#-------------------------------------------------------------------
# Now we are almost ready to perform the bisection and find the root - the angle theta_max 

# We have to define limits where the bisection method will search for the root

#lower limit set equal to sigma. upper limit equal to 50 sigma

lowlimit = 1.1*sigma
uplimit = 20*sigma

theta_max = bisection(func_root, lowlimit, uplimit)

functval = func_root(theta_max)

# convert now theta critical to degrees

theta_maxdeg = theta_max*raddeg

print('Critical angle in rads is:')
print(theta_max)

print('Critical angle in degrees is:')
print(theta_maxdeg)

print('Value of the function is:')
print(functval)




//anaconda/lib/python3.5/site-packages/scipy/integrate/quadpack.py:356: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  warnings.warn(msg, IntegrationWarning)


Entrance no:
0
Entrance no:
1
Entrance no:
2
Entrance no:
3
Entrance no:
4
Entrance no:
5
Entrance no:
6
Entrance no:
7
Entrance no:
8
Entrance no:
9
Entrance no:
10
Entrance no:
11
Entrance no:
12
Entrance no:
13
Entrance no:
14
Entrance no:
15
Entrance no:
16
Entrance no:
17
Entrance no:
18
Entrance no:
19
Entrance no:
20
Entrance no:
21
Entrance no:
22
Entrance no:
23
Entrance no:
24
Entrance no:
25
Entrance no:
26
Entrance no:
27
Entrance no:
28
Entrance no:
29
Entrance no:
30
Entrance no:
31
Entrance no:
32
Entrance no:
33
Entrance no:
34
Entrance no:
35
Entrance no:
36
Entrance no:
37
Entrance no:
38
Entrance no:
39
Critical angle in rads is:
0.008413796007067313
Critical angle in degrees is:
0.482074752571927
Value of the function is:
7.14597270246e-11
